#count,min,max,avg
#group by
#joins

In [0]:
emp_data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',None,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,None,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,None,29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(None,None,None,None,None,None),
(7,'adam',37,65000,'us','IT')
]

sche = ["id","name","age","salary","counry","dept"]

emp_df = spark.createDataFrame(data = emp_data, schema = sche)

emp_df.show()

+----+-------+----+------+-------+-----------+
|  id|   name| age|salary| counry|       dept|
+----+-------+----+------+-------+-----------+
|   1| manish|  26| 20000|  india|         IT|
|   2|  rahul|null| 40000|germany|engineering|
|   3|  pawan|  12| 60000|  india|      sales|
|   4|roshini|  44|  null|     uk|engineering|
|   5|raushan|  35| 70000|  india|      sales|
|   6|   null|  29|200000|     uk|         IT|
|   7|   adam|  37| 65000|     us|         IT|
|   8|  chris|  16| 40000|     us|      sales|
|null|   null|null|  null|   null|       null|
|   7|   adam|  37| 65000|     us|         IT|
+----+-------+----+------+-------+-----------+



In [0]:
emp_df.count()   #RUN act as action

Out[9]: 10

In [0]:
emp_df.select(count("name")).show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3209656271287299>:1
----> 1 emp_df.select(count("name")).show()

NameError: name 'count' is not defined

In [0]:
from pyspark.sql.functions import*

df = emp_df.select(count("name")) #not execuuted


In [0]:
df.show()

+-----------+
|count(name)|
+-----------+
|          8|
+-----------+



In [0]:
emp_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: float (nullable = true)
 |-- counry: string (nullable = true)
 |-- dept: string (nullable = true)



In [0]:
emp_df = emp_df.withColumn("salary", emp_df["salary"].cast("float"))

In [0]:
emp_df.select(
        sum("salary").alias("Total_salary"),
        max("salary"),
        min("salary"),
        avg("salary").alias("Average_sal"),
        count("salary")).show()

+------------+-----------+-----------+-----------+-------------+
|Total_salary|max(salary)|min(salary)|Average_sal|count(salary)|
+------------+-----------+-----------+-----------+-------------+
|    560000.0|   200000.0|    20000.0|    70000.0|            8|
+------------+-----------+-----------+-----------+-------------+



#groupBy

In [0]:
data = [
(1,'manish',26,20000,'india','IT'),
(2,'rahul',23,40000,'germany','engineering'),
(3,'pawan',12,60000,'india','sales'),
(4,'roshini',44,80000,'uk','engineering'),
(5,'raushan',35,70000,'india','sales'),
(6,'grv',29,200000,'uk','IT'),
(7,'adam',37,65000,'us','IT'),
(8,'chris',16,40000,'us','sales'),
(9,'aam',37,65000,'us','IT')
]

schema = ["id","name","age","salary","country","dept"]

df = spark.createDataFrame(data = data, schema = schema)

df.show()

+---+-------+---+------+-------+-----------+
| id|   name|age|salary|country|       dept|
+---+-------+---+------+-------+-----------+
|  1| manish| 26| 20000|  india|         IT|
|  2|  rahul| 23| 40000|germany|engineering|
|  3|  pawan| 12| 60000|  india|      sales|
|  4|roshini| 44| 80000|     uk|engineering|
|  5|raushan| 35| 70000|  india|      sales|
|  6|    grv| 29|200000|     uk|         IT|
|  7|   adam| 37| 65000|     us|         IT|
|  8|  chris| 16| 40000|     us|      sales|
|  9|    aam| 37| 65000|     us|         IT|
+---+-------+---+------+-------+-----------+



In [0]:
df.groupBy("dept").agg(sum("salary"),count("id")).show()

+-----------+-----------+---------+
|       dept|sum(salary)|count(id)|
+-----------+-----------+---------+
|         IT|     350000|        4|
|engineering|     120000|        2|
|      sales|     170000|        3|
+-----------+-----------+---------+



In [0]:
df.createOrReplaceTempView("dept")

In [0]:
spark.sql("""
          with cte as (select name,age,salary,dept,
          sum(salary) over(partition by dept) as dept_sal
          from dept)

          select name,dept,salary,round((salary*100/dept_sal),2) as sal_percenage
          from cte
          
          """).show(
              
          )

+-------+-----------+------+-------------+
|   name|       dept|salary|sal_percenage|
+-------+-----------+------+-------------+
| manish|         IT| 20000|         5.71|
|    grv|         IT|200000|        57.14|
|   adam|         IT| 65000|        18.57|
|    aam|         IT| 65000|        18.57|
|  rahul|engineering| 40000|        33.33|
|roshini|engineering| 80000|        66.67|
|  pawan|      sales| 60000|        35.29|
|raushan|      sales| 70000|        41.18|
|  chris|      sales| 40000|        23.53|
+-------+-----------+------+-------------+



#Join

In [0]:

customer_data = [(1,'manish','patna',"30-05-2022"),
(2,'vikash','kolkata',"12-03-2023"),
(3,'nikita','delhi',"25-06-2023"),
(4,'rahul','ranchi',"24-03-2023"),
(5,'mahesh','jaipur',"22-03-2023"),
(6,'prantosh','kolkata',"18-10-2022"),
(7,'raman','patna',"30-12-2022"),
(8,'prakash','ranchi',"24-02-2023"),
(9,'ragini','kolkata',"03-03-2023"),
(10,'raushan','jaipur',"05-02-2023")]

customer_schema=['customer_id','customer_name','address','date_of_joining']

customer = spark.createDataFrame(data = customer_data,schema = customer_schema)

customer.show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          1|       manish|  patna|     30-05-2022|
|          2|       vikash|kolkata|     12-03-2023|
|          3|       nikita|  delhi|     25-06-2023|
|          4|        rahul| ranchi|     24-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|
|          6|     prantosh|kolkata|     18-10-2022|
|          7|        raman|  patna|     30-12-2022|
|          8|      prakash| ranchi|     24-02-2023|
|          9|       ragini|kolkata|     03-03-2023|
|         10|      raushan| jaipur|     05-02-2023|
+-----------+-------------+-------+---------------+



In [0]:

sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

sales_schema=['customer_id','product_id','quantity','date_of_purchase']

sales = spark.createDataFrame(data = sales_data,schema = sales_schema)

sales.show()

+-----------+----------+--------+----------------+
|customer_id|product_id|quantity|date_of_purchase|
+-----------+----------+--------+----------------+
|          1|        22|      10|      01-06-2022|
|          1|        27|       5|      03-02-2023|
|          2|         5|       3|      01-06-2023|
|          5|        22|       1|      22-03-2023|
|          7|        22|       4|      03-02-2023|
|          9|         5|       6|      03-03-2023|
|          2|         1|      12|      15-06-2023|
|          1|        56|       2|      25-06-2023|
|          5|        12|       5|      15-04-2023|
|         11|        12|      76|      12-03-2023|
+-----------+----------+--------+----------------+



In [0]:
product_data = [(1, 'fanta',20),
(2, 'dew',22),
(5, 'sprite',40),
(7, 'redbull',100),
(12,'mazza',45),
(22,'coke',27),
(25,'limca',21),
(27,'pepsi',14),
(56,'sting',10)]

product_schema=['id','name','price']

product = spark.createDataFrame(data = product_data,schema = product_schema)

product.show()

+---+-------+-----+
| id|   name|price|
+---+-------+-----+
|  1|  fanta|   20|
|  2|    dew|   22|
|  5| sprite|   40|
|  7|redbull|  100|
| 12|  mazza|   45|
| 22|   coke|   27|
| 25|  limca|   21|
| 27|  pepsi|   14|
| 56|  sting|   10|
+---+-------+-----+



In [0]:
customer.join(sales,customer["customer_id"] == sales["customer_id"],"inner").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15

In [0]:
customer.join(sales,customer["customer_id"] == sales["customer_id"],"left").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          3|       nikita|  delhi|     25-06-2023|       null|      null|    null|            null|
|          4|        rahul| ranchi|     24-03-2023|       null|      null|    null|        

In [0]:
customer.join(sales,customer["customer_id"] == sales["customer_id"],"right").show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          7|        raman|  patna|     30-12-2022|          7|        22|       4|      03-02-2023|
|          9|       ragini|kolkata|     03-03-2023|          9|         5|       6|      03-03-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15

In [0]:
customer.join(sales,customer["customer_id"] == sales["customer_id"],"inner").select("customer_id").show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-3209656271287320>:1
----> 1 customer.join(sales,customer["customer_id"] == sales["customer_id"],"inner").select("customer_id").show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type: ignore[misc]
   2979     """Projects a set of expressions and returns a new :class:`DataFrame`.
   2980 
   2981     .. versionadded:: 1.3.0
   (...)
   3021  

In [0]:
customer.join(sales,customer["customer_id"] == sales["customer_id"],"inner")\
    .select(sales["customer_id"]).show()

+-----------+
|customer_id|
+-----------+
|          1|
|          1|
|          1|
|          2|
|          2|
|          5|
|          5|
|          7|
|          9|
+-----------+



#Combining more than 2 tables

In [0]:
df = customer.join(sales,customer["customer_id"] == sales["customer_id"],"inner")\
    .select(sales["product_id"])

df.show()

+----------+
|product_id|
+----------+
|        22|
|        27|
|        56|
|         5|
|         1|
|        22|
|        12|
|        22|
|         5|
+----------+



In [0]:
product.show()

+---+-------+-----+
| id|   name|price|
+---+-------+-----+
|  1|  fanta|   20|
|  2|    dew|   22|
|  5| sprite|   40|
|  7|redbull|  100|
| 12|  mazza|   45|
| 22|   coke|   27|
| 25|  limca|   21|
| 27|  pepsi|   14|
| 56|  sting|   10|
+---+-------+-----+



In [0]:
df.join(product,df["product_id"]==product["id"], "inner").show()

+----------+---+------+-----+
|product_id| id|  name|price|
+----------+---+------+-----+
|         1|  1| fanta|   20|
|         5|  5|sprite|   40|
|         5|  5|sprite|   40|
|        12| 12| mazza|   45|
|        22| 22|  coke|   27|
|        22| 22|  coke|   27|
|        22| 22|  coke|   27|
|        27| 27| pepsi|   14|
|        56| 56| sting|   10|
+----------+---+------+-----+



In [0]:
df = df.distinct()

In [0]:
df.join(product,df["product_id"]==product["id"], "inner").show()

+----------+---+------+-----+
|product_id| id|  name|price|
+----------+---+------+-----+
|         1|  1| fanta|   20|
|         5|  5|sprite|   40|
|        12| 12| mazza|   45|
|        22| 22|  coke|   27|
|        27| 27| pepsi|   14|
|        56| 56| sting|   10|
+----------+---+------+-----+



In [0]:
df.join(product,df["product_id"]==product["id"], "outer").show()

+----------+---+-------+-----+
|product_id| id|   name|price|
+----------+---+-------+-----+
|         1|  1|  fanta|   20|
|      null|  2|    dew|   22|
|         5|  5| sprite|   40|
|      null|  7|redbull|  100|
|        12| 12|  mazza|   45|
|        22| 22|   coke|   27|
|      null| 25|  limca|   21|
|        27| 27|  pepsi|   14|
|        56| 56|  sting|   10|
+----------+---+-------+-----+



In [0]:
df.join(product,df["product_id"]==product["id"], "left_anti").show()

+----------+
|product_id|
+----------+
+----------+



In [0]:
product.join(df,df["product_id"]==product["id"], "left_anti").show()

+---+-------+-----+
| id|   name|price|
+---+-------+-----+
|  2|    dew|   22|
|  7|redbull|  100|
| 25|  limca|   21|
+---+-------+-----+



In [0]:
emp_data = [(1,'manish',50000,'IT','m'),
(2,'vikash',60000,'sales','m'),
(3,'raushan',70000,'marketing','m'),
(4,'mukesh',80000,'IT','m'),
(5,'priti',90000,'sales','f'),
(6,'nikita',45000,'marketing','f'),
(7,'ragini',55000,'marketing','f'),
(8,'rashi',100000,'IT','f'),
(9,'aditya',65000,'IT','m'),
(10,'rahul',50000,'marketing','m'),
(11,'rakhi',50000,'IT','f'),
(12,'akhilesh',90000,'sales','m')]